In [8]:
import glob
import re
from collections import Counter
import json
from subprocess import Popen, PIPE
import os

In [ ]:
# Read all logFiles for all libraries
logFiles = {}
for f in glob.glob('/home/XYZ/docker/attached_dir/testing-results/seeds30k/logs/*'):
    with open(f, "r", errors="replace") as fileO:
        logFiles[(f.replace("output.", "").rsplit("/logs/", 1)[1].split(".")[0] + "-library-" + f.rsplit("-", 1)[1].split(".", 1)[0])] = fileO.readlines()
print(len(logFiles))

In [ ]:
# Only consider logFiles for whom at least one library passed and at least one failed
uniqueDiscrepancies = {}
allFiles = 0

dirsList = ["/home/XYZ//docker/attached_dir/testing-results/seeds30k/json/*"]
filesList = []
for x in dirsList:
    for f in glob.glob(x):
        filesList.append(f)

for f in filesList:
    with open(f, "r") as fileO:
        if "new" not in f:
            fileNumber = f.split(".pem")[0].rsplit("/", 1)[1]
            # for 30k Transcert
            # if int(fileNumber) > 30000:
            #     continue
        
        allFiles += 1
        
        data = json.load(fileO)
        this_pattern = ""
        for elem in data:
            this_pattern += str((int(data[elem] != 0)))
        
        if this_pattern == "00000" or this_pattern == "11111":
            continue
        
        if this_pattern not in uniqueDiscrepancies:
            uniqueDiscrepancies[this_pattern] = set()
            
        uniqueDiscrepancies[this_pattern].add((f.rsplit("/", 1)[1].replace("output.", "").split(".")[0]))

print("Total result files used: ", allFiles)
print("Total discrepancies: ", len(uniqueDiscrepancies))
print("\n")
for discrepancy in uniqueDiscrepancies:
    print(discrepancy, len(uniqueDiscrepancies[discrepancy]))

In [16]:
def get_logfile_output_stripped(output_lines, lib):
    allLines = ""

    if "libressl" in lib:
        for l in output_lines: 
            l = re.sub(r'/cert_data_pem/.*pem', '', l)
            l = re.sub(r'[^ ]*:error:[^ ]*:', ':error:', l)
            
            if " = " in l: 
                continue

            allLines += l + "\n"
    
    elif "openssl" in lib:
        for l in output_lines: 
            l = re.sub(r'/cert_data_pem/.*pem', '', l)
            l = re.sub(r'[^ ]*:error:[^ ]*:', ':error:', l)
            
            if " = " in l: 
                continue
            
            allLines += l + "\n"        
            
    elif "gnutls" in lib:
        for l in output_lines:            
            if "=" in l:
                continue
                
            if "Subject: " in l or "Issuer: " in l:
                continue
            
            if "Signature algorithm: " in l:
                continue
            
            if "Note that no verification profile was selected" in l:
                continue
            
            if "Use --verify-profile low to apply the default" in l:
                continue
            
            if "sestaip-sbreb1" in l or "G'7F\x16Ɛ\n" in l:
                continue

            allLines += l + "\n"        
        
    elif "matrixssl" in lib:
        for l in output_lines:            
            l = re.sub(r'/cert_data_pem/.*pem', '', l)
            
            l = re.sub(r'\[111\]:ANF Secure Server Root CA', '', l)
            l = re.sub(r'\[[0-9]*\]:', '*****', l)
            l = re.sub(r'Validate:(.)*FAIL', "Validate **** FAIL", l) 
                        
            if "WARN subject not provided, SUBJ validation will be skipped" in l:
                continue 
                
            if "*****" in l and "Validate" not in l:
                continue
            
            if "Loaded chain file" in l:
                continue 
                
            if "Loaded root file" in l:
                continue
                            
            if "oCA Certificate" in l or "ED\n" == l or " G1\n" == l:
                continue
            
            # For AddressSanitizer
            l = re.sub(r'==[0-9]*==ERROR', '', l)

            if "#" in l:
                continue 

            if "SUMMARY: AddressSanitizer" in l:
                continue
            
            if "Direct leak of" in l:
                continue
            
            allLines += l + "\n"
        
    
    elif "mbedtls" in lib:
        for l in output_lines:            
            if ":" in l:
                continue

            allLines += l + "\n"
    
    else:
        print("No match")
        assert False == True
    
    return allLines

In [ ]:
libraries = {"openssl", "libressl", "gnutls", "mbedtls", "matrixssl"}
hashesToOriginal = {}
uniqueOutputsPerDiscrepancy = {}

for uniqueDiscrepancy in uniqueDiscrepancies:
    uniqueOutputsPerDiscrepancy[uniqueDiscrepancy] = {}

    for disc_file in uniqueDiscrepancies[uniqueDiscrepancy]:
        output = ""
        for lib in libraries:
            output += "\n%%%%\n" + lib.upper() + "\n%%%%\n" + get_logfile_output_stripped(logFiles[(disc_file + "-library-" + lib)], lib)
        
        output = re.sub("\s\s+" , " ", output)
        hashesToOriginal[str(hash(output))] = output
        output = str(hash(output))
        if output not in uniqueOutputsPerDiscrepancy[uniqueDiscrepancy]:
            uniqueOutputsPerDiscrepancy[uniqueDiscrepancy][output] = set()

        uniqueOutputsPerDiscrepancy[uniqueDiscrepancy][output].add(disc_file)

In [174]:
print("Discrepancy", "(", "Number of certificates", ")", "\n")
for x in uniqueOutputsPerDiscrepancy:
    print(x, "(", len(uniqueDiscrepancies[x]), ")", "\n")
    print("   Further groups: ", len(uniqueOutputsPerDiscrepancy[x]))
    output = "("
    for y in uniqueOutputsPerDiscrepancy[x]:
        output += " " + str(len(uniqueOutputsPerDiscrepancy[x][y]))
    print("   ", output + ")", "\n\n")

Discrepancy ( Number of certificates ) 

00001 ( 103828 ) 

   Further groups:  48
    ( 85594 4429 13541 64 20 31 24 44 30 1 1 1 1 1 1 1 9 1 1 1 1 1 1 1 1 1 1 1 1 1 3 1 1 1 1 1 1 3 1 1 1 1 1 1 1 1 1 1) 


01011 ( 2795 ) 

   Further groups:  34
    ( 1992 639 1 9 10 17 2 2 5 47 13 16 1 1 6 4 1 2 2 4 2 3 1 4 1 1 2 1 1 1 1 1 1 1) 


00011 ( 5378 ) 

   Further groups:  20
    ( 1334 219 1328 597 1182 91 239 102 176 8 14 47 25 1 10 1 1 1 1 1) 


11001 ( 12225 ) 

   Further groups:  553
    ( 1287 1567 299 20 196 1863 332 77 577 697 19 72 14 92 25 11 126 137 102 681 60 1 17 5 18 24 7 147 34 34 99 178 9 46 31 133 82 113 7 16 11 143 63 1 65 48 1 21 29 5 70 2 25 13 21 19 1 10 20 20 46 98 29 1 44 9 23 18 53 1 1 16 15 7 20 1 23 3 3 2 11 51 181 3 42 15 29 1 25 1 28 8 1 44 36 98 4 3 12 14 28 21 7 17 2 10 8 17 1 65 1 15 1 28 8 2 2 23 20 27 16 2 16 24 3 5 1 2 1 23 7 26 9 1 4 9 39 8 1 6 17 3 1 2 1 5 1 1 5 6 12 1 5 1 4 3 16 8 2 1 1 1 2 15 4 2 3 4 8 2 1 7 1 9 3 7 1 1 1 4 10 2 1 1 2 1 6 1 2 3 14 10 1

In [26]:
import OpenSSL
import ssl
from datetime import datetime

def get_validity_dates(file_path):
    try:
        with open(file_path) as cert:
            x509 = OpenSSL.crypto.load_certificate(OpenSSL.crypto.FILETYPE_PEM, cert.read())
            return datetime.strptime(x509.get_notBefore()[:-7].decode(), '%Y%m%d'), datetime.strptime(x509.get_notAfter()[:-7].decode(), '%Y%m%d')
    except:
        print("Error parsing cert time")
        
def get_version(file_path):
    try:
        with open(file_path) as cert:
            x509 = OpenSSL.crypto.load_certificate(OpenSSL.crypto.FILETYPE_PEM, cert.read())
            return x509.get_version()
    except:
        print("Error parsing version")
        
def get_subject(file_path):
    try:
        with open(file_path) as cert:
            x509 = OpenSSL.crypto.load_certificate(OpenSSL.crypto.FILETYPE_PEM, cert.read())
            return x509.get_subject()
    except:
        print("Error parsing subject")

In [48]:
# Given an input string, find all discrepancy groups and subgroups with that text in the output

check_strings = [
    # GNUTLS
    "signature in the certificate is invalid",
    "certificate chain uses insecure algorithm",
    "certificate issuer is unknown",
    "certificate chain uses expired certificate",
    "Error in the time fields of certificate",
    "Duplicate extension in X.509 certificate",
    "certificate chain violates the signer's constraints",
    "certificate chain uses not yet valid certificate",
    "certificate contains an unknown critical extension",
    
    # LIBRESSL
    "certificate signature failure",
    "unable to get local issuer certificate",
    "unsupported or invalid name syntax",
    "certificate has expired",
    "EE certificate key too weak",
    "certificate is not yet valid",
    "unhandled critical extension",
    "bad signature",
    "invalid time format",
    ### "no subject details",
    ### "wrong signature length",
    ### "unknown message digest algorithm",
    "unknown signature algorithm",
    ## "Unspecified certificate verification error",
    "invalid or inconsistent certificate policy extension"
    
    # MBEDTLS
    "certificate is not correctly signed by the trusted CA",
    "certificate is signed with an unacceptable hash",
    "certificate validity has expired",
    "certificate validity starts in the future",
    
    # MATRIXSSL
    "FAIL Auth Key / Subject Key Match",
    "FAIL Distinguished Name Match",
    "Assertion `faildate == 0' failed",
    "unsupported critical extension was encountered",
    "End-entity certificate not for TLS usage",
 
    # OPENSSL
    "certificate signature failure",
    "bad signature",
    "certificate has expired",
    "format error in certificate's notBefore field",
    "format error in certificate's notAfter field",
    "certificate is not yet valid",
    "unhandled critical extension",
    ### "unknown message digest algorithm",   
]

show_expiry = [
               #  "certificate is not yet valid", 
                "format error in certificate's notAfter field", 
                "format error in certificate's notBefore field",
               # "certificate has expired",
               
               # "certificate validity starts in the future",
               # "certificate validity has expired",
               
                # "certificate has expired",
                # "certificate is not yet valid",
                "invalid time format",
               
                # "certificate chain uses expired certificate",
                "Error in the time fields of certificate",
                # "certificate chain uses not yet valid certificate",
              ]

show_version = [
    "unhandled critical extension",
    "unsupported critical extension was encountered",
    "certificate contains an unknown critical extension",
    "Duplicate extension in X.509 certificate",
]

issuer_unknown = [
        "certificate issuer is unknown",
        "unable to get local issuer certificate",
]

#check_string = "certificate is not yet valid".lower()
check_string = "expired".lower()
#check_string = "unsupported critical extension was encountered".lower()

# Show discrepancies where at least some string match is available
for c_string in show_expiry:
    print("\n******\n")
    print(c_string)
    unique_xs = set()
    for x in uniqueOutputsPerDiscrepancy:
        for y in uniqueOutputsPerDiscrepancy[x]:
            if c_string.lower() in hashesToOriginal[y].lower():
                print(x, y, len(uniqueOutputsPerDiscrepancy[x][y]))
                unique_xs.add(x)
                # print(hashesToOriginal[y])
    
                # Show cert expiry if needed for debugging
                # if c_string in show_expiry:
                #     for sample_cert in uniqueOutputsPerDiscrepancy[x][y]:
                #         print(get_validity_dates("//home/XYZ/docker/attached_dir/cert_data_pem/v3-experiments-extra/zmap-data-1024-3-0.0002lr-0.1dropout-epoch3-step300000T1.5/output." + sample_cert + ".pem"))
                #         break
                        
                # if c_string in show_version:
                #     sample_cert = next(iter(uniqueOutputsPerDiscrepancy[x][y]))
                #     print(get_version("//home/XYZ/docker/attached_dir/cert_data_pem/v3-experiments-extra/zmap-data-1024-3-0.0002lr-0.1dropout-epoch3-step300000T1.5/output." + sample_cert + ".pem"))
                    
    print(unique_xs)
    
#Show discrepancies where the library logs does not include any relevant strings
# for x in uniqueOutputsPerDiscrepancy:
#     print("\n******\n")
#     for y in uniqueOutputsPerDiscrepancy[x]:        
#         any = False
#         for c_string in check_strings:
#             if c_string.lower() in hashesToOriginal[y].lower():
#                 #print(x, y, c_string)
#                 any = True
#                 break
        
#         if not any:
#             print(x, y)  


******

format error in certificate's notAfter field
11011 -3187906424609888014 1
11011 -2657137207487859028 1
11011 2235400425619931573 1
11011 1584486032781325409 1
11011 -8522345408203629903 1
11011 -3115074770409586240 12
11011 -808926891530323919 1
11011 3505522337464636842 1
11011 875542990026351046 1
11011 5488660927273692014 1
11011 -8719938231370990943 1
11011 -8583399422301707541 1
11011 9044453024989934453 1
11011 1285493957703757489 1
11011 -2169540945237679806 3
11011 -7221515120981548700 1
11011 279040941755117085 1
11011 -5501192719338077867 1
11011 8231595846126776567 1
11011 2851257836780697133 1
11011 6260710259810901297 1
11011 8391250168685297603 1
11011 -4709149861052726513 1
11011 369420708621130203 1
11011 -3282940991872554065 1
11011 9192950369686526931 1
11011 -1572246033448015962 1
11011 1494153565186099832 1
11011 1104378153935942287 1
11011 7003420533134511833 1
11011 -6750245380121633190 1
11011 -501078992575287867 1
11011 -7102071544005444717 1
11011 -489

In [50]:
print(len(uniqueOutputsPerDiscrepancy["11011"]["8391250168685297603"]), uniqueOutputsPerDiscrepancy["11011"]["8391250168685297603"])
#print(hashesToOriginal["-7264588278289527287"])
# print("HI")
# import random

# x = 0
# while True:
# #     x_disc = random.choice(list(uniqueOutputsPerDiscrepancy.keys()))
#     x_disc = "10001"
    
#     for y_output in uniqueOutputsPerDiscrepancy[x_disc]:
# #     y_output = random.choice(list(uniqueOutputsPerDiscrepancy[x_disc].keys()))
#         #print(x, x_disc, y_output, len(uniqueOutputsPerDiscrepancy[x_disc][y_output]), uniqueOutputsPerDiscrepancy[x_disc][y_output])
#         #print(uniqueOutputsPerDiscrepancy[x_disc][y_output])
#         #print(hashesToOriginal[y_output])
#         x += 1

#     # break

1 {'04d15059-d921-4175-bc8b-eed7a9dc8d3a'}
